In [24]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# 3. Exchange economy

Consider an **exchange economy** with

1. 3 goods, $(x_1,x_2,x_3)$
2. $N$ consumers indexed by \\( j \in \{1,2,\dots,N\} \\)
3. Preferences are Cobb-Douglas with log-normally distributed coefficients

    $$ \begin{eqnarray*}
    u^{j}(x_{1},x_{2},x_{3}) &=& 
    \left(x_{1}^{\beta_{1}^{j}}x_{2}^{\beta_{2}^{j}}x_{3}^{\beta_{3}^{j}}\right)^{\gamma}\\
     &  & \,\,\,\beta_{i}^{j}=\frac{\alpha_{i}^{j}}{\alpha_{1}^{j}+\alpha_{2}^{j}+\alpha_{3}^{j}} \\
     &  & \,\,\,\boldsymbol{\alpha}^{j}=(\alpha_{1}^{j},\alpha_{2}^{j},\alpha_{3}^{j}) \\ 
     &  & \,\,\,\log(\boldsymbol{\alpha}^j) \sim \mathcal{N}(\mu,\Sigma) \\
    \end{eqnarray*} $$

4. Endowments are exponentially distributed,

$$
\begin{eqnarray*}
\boldsymbol{e}^{j} &=& (e_{1}^{j},e_{2}^{j},e_{3}^{j}) \\
 &  & e_i^j \sim f, f(z;\zeta) =  1/\zeta \exp(-z/\zeta)
\end{eqnarray*}
$$

Let $p_3 = 1$ be the **numeraire**. The implied **demand functions** are:

$$
\begin{eqnarray*}
x_{i}^{\star j}(p_{1},p_{2},\boldsymbol{e}^{j})&=&\beta^{j}_i\frac{I^j}{p_{i}} \\
\end{eqnarray*}
$$

where consumer $j$'s income is

$$I^j = p_1 e_1^j + p_2 e_2^j +p_3 e_3^j$$

The **parameters** and **random preferences and endowments** are given by:

In [2]:
# a. parameters
N = 50000
mu = np.array([3,2,1])
Sigma = np.array([[0.25, 0, 0], [0, 0.25, 0], [0, 0, 0.25]])
gamma = 0.8
zeta = 1

# b. random draws
seed = 1986
np.random.seed(seed)

# preferences
alphas = np.exp(np.random.multivariate_normal(mu, Sigma, size=N))
betas = alphas/np.reshape(np.sum(alphas,axis=1),(N,1))

# endowments
e1 = np.random.exponential(zeta,size=N)
e2 = np.random.exponential(zeta,size=N)
e3 = np.random.exponential(zeta,size=N)

**Question 1:** Plot the histograms of the budget shares for each good across agents.

To plot the budget shares, we need to define each individual budget share $\beta_i$ from betas, and plot these in a figure each

In [3]:
#First we define beta one, two and three (Index starts with 0)
beta_1 = betas[:,0]
beta_2 = betas[:,1]
beta_3 = betas[:,2]

In [9]:
#Secondly we define a DataFrame with each beta
Budget_share = pd.DataFrame([beta_1,beta_2,beta_3])

#And transpose
Budget_share = pd.DataFrame(Budget_share).transpose()

#Rename the coloumns
Budget_share = pd.DataFrame(Budget_share).rename(columns={0:"beta_1",1:"beta_2",2:"beta_3"})

In [51]:
#Define a functions that creates the figure for each beta. 
def budget_share(beta_value):
    pd.DataFrame(Budget_share).hist(column=beta_value,bins=100)
    plt.xlabel('Budget share value')
    plt.ylabel('Density')
    plt.title(f'Budget share across agents for {str(beta_value)}')
    plt.grid(True)
    return plt.show()

In [52]:
#Create an interactive figure that shows the distribution for each beta
widgets.interact(budget_share,
                 beta_value = widgets.Dropdown(options=["beta_1","beta_2","beta_3"]))

interactive(children=(Dropdown(description='beta_value', options=('beta_1', 'beta_2', 'beta_3'), value='beta_1…

<function __main__.budget_share(beta_value)>

Comments for the histograms:
It appears that the agents have a preference for good one, as beta_1 has the largest expected value, even though it also has a large spread. Good three is generally not preferred with a very low expected value and low spread. Finally good two has a low expected value, but also a relatively large spread.

Consider the **excess demand functions:**

$z_i(p_1,p_2) = \sum_{j=1}^N x_{i}^{\star j}(p_{1},p_{2},\boldsymbol{e}^{j}) - e_i^j$

**Question 2:** Plot the excess demand functions.

**Quesiton 3:** Find the Walras-equilibrium prices, $(p_1,p_2)$, where both excess demands are (approximately) zero, e.g. by using the following tâtonnement process:

1. Guess on $p_1 > 0$, $p_2 > 0$ and choose tolerance $\epsilon > 0$ and adjustment aggressivity parameter, $\kappa > 0$.
2. Calculate $z_1(p_1,p_2)$ and $z_2(p_1,p_2)$.
3. If $|z_1| < \epsilon$ and $|z_2| < \epsilon$ then stop.
4. Else set $p_1 = p_1 + \kappa \frac{z_1}{N}$ and $p_2 = p_2 + \kappa \frac{z_2}{N}$ and return to step 2.

**Question 4:** Plot the distribution of utility in the Walras-equilibrium and calculate its mean and variance.

**Question 5:** Find the Walras-equilibrium prices if instead all endowments were distributed equally. Discuss the implied changes in the distribution of utility. Does the value of $\gamma$ play a role for your conclusions?

In [ ]:
#First we define the demand functions
def demand_func1(beta_value,p1,p2):
    I = p1*e1+p2*e2+e3
    return betas[beta_value]*I/p1
def demand_func2(betas,p1,p2):
    I = p1*e1+p2*e2+e3
    return betas[beta_value]*I/p2
def demand_func3(betas, p1,p2):
    I = p1*e1+p2*e2+e3
    return betas[beta_value]*I/1